In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold 
dados3= pd.read_excel('queen_por_pontuacao.xlsx', sheet_name=0)
queen_por_posicao= pd.read_excel('queen_por_pontuacao.xlsx', sheet_name=0)

### ADICIONA UM DF COM O NUMERO DE EPISODIOS, E METADE DOS EPISODIOS DAS TEMPORADAS ###

In [ ]:
#
agrupa_maior_ep=dados3.groupby("tempfranquia")["ep"].max().reset_index()
#agrupa_maior_ep


maxep_metadeep = agrupa_maior_ep.rename(columns={"ep": "maxep"})
maxep_metadeep["metadeep"] = (maxep_metadeep["maxep"] / 2).astype(int)
maxep_metadeep.head()


,tempfranquia,maxep,metadeep
0,101,6,3
1,102,7,3
2,103,9,4
3,104,9,4
4,105,7,3


In [ ]:
## inner join dos dados com a dim queen para adicionar colunas necessarias

dados_meio= pd.merge(
    dados3,
    maxep_metadeep,
    left_on=["tempfranquia", "ep"],
    right_on=["tempfranquia", "metadeep"],
    how="inner"
)
dados_meio=dados_meio.drop(['ep', 'metadeep', 'maxep','colocacao'], axis=1)
dados_meio.head()

,id,queen,bom,ruim,media,idade,tempfranquia,vencedora
0,1,BeBe Zahara Benet,8,2,2.0,28,101,1
1,2,Nina Flowers,14,0,4.0,34,101,0
2,3,Rebecca Glasscock,8,4,1.0,26,101,0
3,4,Shannel,5,6,-1.0,29,101,0
4,5,Ongina,14,0,4.0,26,101,0


In [17]:
df = dados_meio.copy()
features = ['bom', 'ruim', 'media', 'idade']
X = df[features]
y = df['vencedora']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [19]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Regressão 
logreg = LogisticRegression(class_weight='balanced', random_state=42)
logreg.fit(X_train_scaled, y_train)

y_pred_lr = logreg.predict(X_test_scaled)
probabilidades_vencedora = logreg.predict_proba(X_test_scaled)

print("Logistic Regression")
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

Logistic Regression
              precision    recall  f1-score   support

           0       1.00      0.77      0.87        83
           1       0.37      1.00      0.54        11

    accuracy                           0.80        94
   macro avg       0.68      0.89      0.70        94
weighted avg       0.93      0.80      0.83        94

[[64 19]
 [ 0 11]]


In [ ]:
probabilidades = probabilidades_vencedora[:, 1]

queens_do_teste = dados_meio.loc[X_test.index, 'queen']


resultados = pd.DataFrame({
    'queen': queens_do_teste,
    'Chance de Vencer (%)': probabilidades * 100
})


resultados['Chance de Vencer (%)'] = resultados['Chance de Vencer (%)'].round(2)

print("--- Chance de cada rainha ganhar ---")
print(resultados.sort_values(by='Chance de Vencer (%)', ascending=False))

--- Chance de cada rainha ganhar ---
                         queen  Chance de Vencer (%)
120  Angeria Paris VanMichaels                 93.02
25               Chad Michaels                 90.50
99                  Gigi Goode                 89.61
201                   Organzza                 86.18
406                    Khianna                 85.07
..                         ...                   ...
303                 Karna Ford                  0.86
274              Elektra Shock                  0.77
190          Miss Naomi Carter                  0.66
469                   Shortgun                  0.34
59                    Kandy Ho                  0.03

[94 rows x 2 columns]
